# Sparkle Motion Movie Machine


This notebook **only** supports the Google Colab A100 runtime. Every cell assumes you are in `/content` with a GPU attached and that the repo lives at `/content/sparkle_motion`.
- Step through the cells in order; do not skip the install step.
- All workspace data stays under `/content/<workspace_name>`; there is a single storage path for every run.
- The smoke-test cell is a safe stub that checks imports and shows how to run the orchestrator in simulation mode once the stack is installed.

## Notes and expectations
- Always request an A100 runtime in Colab (`Runtime → Change runtime type → GPU → A100`).
- Keep the notebook tab in focus while installs run; cancel/restart the runtime if `pip install` fails.
- Everything installs under `/content`; rerunning the notebook in a fresh runtime recreates the same layout automatically.

In [ ]:
from __future__ import annotations

# Cell 0: Guard execution environment and enforce repo clone
import importlib.util

if importlib.util.find_spec("google.colab") is None:
    raise RuntimeError("This notebook only runs on Google Colab with an A100 GPU attached.")
else:
    print("Running on Google Colab with an A100 GPU.")


Running on Google Colab with an A100 GPU.


In [ ]:
from google.colab import output
import ipywidgets as widgets  # already installed with our setup cells

output.enable_custom_widget_manager()


## Pull the repository into this runtime
Use the next cell when you open this notebook directly from GitHub in Colab. It clones (or updates) the `sparkle_motion` repo inside `/content` so every other helper has access to the full source tree. All paths in the rest of the notebook assume the repo lives at `/content/sparkle_motion`.

In [3]:
!cd /content && git clone https://github.com/ekkus93/sparkle_motion.git

Cloning into 'sparkle_motion'...
remote: Enumerating objects: 3967, done.
remote: Counting objects: 100% (435/435), done.
remote: Compressing objects: 100% (307/307), done.
remote: Total 3967 (delta 260), reused 259 (delta 127), pack-reused 3532 (from 1)
Receiving objects: 100% (3967/3967), 1.57 MiB | 20.05 MiB/s, done.
Resolving deltas: 100% (2202/2202), done.


In [ ]:
# Cell 1: Clone or update the sparkle_motion repo (Colab-friendly)
import os
from pathlib import Path

REPO_ROOT = Path("/content/sparkle_motion").resolve()
globals()["REPO_ROOT"] = REPO_ROOT
os.environ["SPARKLE_MOTION_REPO_ROOT"] = str(REPO_ROOT)
print(f"Working directory switched to {REPO_ROOT}")


Working directory switched to /content/sparkle_motion


In [ ]:
# Cell 1b: ensure repo src directory is importable
import sys
from pathlib import Path

if "REPO_ROOT" not in globals():
    raise RuntimeError("REPO_ROOT is undefined. Run the repo clone cell before configuring sys.path.")

_src_path = Path(globals()["REPO_ROOT"]) / "src"
if str(_src_path) not in sys.path:
    sys.path.insert(0, str(_src_path))
    print(f"Added {_src_path} to sys.path")
else:
    print(f"{_src_path} already present on sys.path")


In [5]:
# Cell 5: Install requirements from requirements-ml.txt
import subprocess
import sys
from pathlib import Path

if "REPO_ROOT" not in globals():
    raise RuntimeError("REPO_ROOT is undefined. Run the repo clone cell before installing dependencies.")

req_path = Path(globals()["REPO_ROOT"]).resolve() / "requirements-ml.txt"
if not req_path.exists():
    raise FileNotFoundError(f"requirements-ml.txt not found at {req_path}")

print("Installing Python dependencies (this may take several minutes)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(req_path)])
print("Dependency install complete.")

Installing Python dependencies (this may take several minutes)...
Dependency install complete.


## Configure workspace and load `.env`
This cell creates your workspace folder under `/content/<workspace_name>`, records the Hugging Face model list, and loads secrets from the repo checkout (`/content/sparkle_motion/.env`). Before running it, create `/content/sparkle_motion/.env` (same folder as the cloned repo) and drop your keys there. The cell will refuse to continue until that file exists and will print the absolute path it just loaded.

### Required entries in `/content/sparkle_motion/.env`
| Key | Purpose | Example value | Notes |
| --- | --- | --- | --- |
| `ADK_PROJECT` | ADK project slug | `sparkle-motion` | Used by the CLI + FunctionTools. |
| `ADK_API_KEY` | API key for that project | `sk-************************` | Follow the steps below to create it. |
| `HF_TOKEN` | Hugging Face token | `hf_************************` | Needed for private model downloads; leave blank if all repos are public. |
| `SPARKLE_DB_PATH` | SQLite path for RunRegistry + dedupe | `/content/SparkleMotion/sparkle.db` | Any absolute path works; the parent folder must exist. Run `touch /content/SparkleMotion/sparkle.db` once if the file is missing. |
| `ARTIFACTS_DIR` | Root folder where agents publish artifacts | `/content/SparkleMotion/artifacts` | Create the directory up front (`mkdir -p /content/SparkleMotion/artifacts`). The CLI + FunctionTools will write manifests/files here. |
| `GOOGLE_ADK_PROFILE` | ToolRegistry profile to use | `local-colab` | Matches the endpoints defined in `configs/tool_registry.yaml`. Keep this if you are running inside Colab. |

Feel free to add other knobs (e.g., `SPARKLE_RECENT_INDEX_SQLITE`, `SMOKE_*` flags) but the table above is the minimum set the notebook expects.

How to create an `ADK_API_KEY`:
1. Visit https://aistudio.google.com and sign in with the project owner account.
2. Open **Projects → sparkle-motion** (or your project).
3. Click **API Keys → Create key**, give it a label, and copy the token.
4. Edit `/content/sparkle_motion/.env` and add `ADK_API_KEY=<copied token>`.

Once the file exists, rerun the cell anytime—it always loads from `/content/sparkle_motion/.env` and keeps the workspace rooted in `/content`.

In [ ]:
# Helper: ensure python-dotenv is installed in this runtime
import importlib.util
import subprocess
import sys

if importlib.util.find_spec("dotenv") is None:
    print("Installing python-dotenv ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
else:
    print("python-dotenv already installed.")


In [ ]:
# Cell 3: Configure workspace and load secrets
import os
from pathlib import Path
from typing import List

from dotenv import load_dotenv

if "REPO_ROOT" not in globals():
    raise RuntimeError("REPO_ROOT is undefined. Run the repo clone cell before configuring the workspace.")

REPO_ROOT = Path(globals()["REPO_ROOT"]).resolve()

WORKSPACE_NAME = "SparkleMotion"          # Folder created under /content
HF_MODELS: List[str] = [
    "stabilityai/stable-diffusion-xl-base-1.0",
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    "Wan-AI/Wan2.1-I2V-14B-720P",
    "Wan-AI/Wan2.1-FLF2V-14B-720P-diffusers",
    "ResembleAI/chatterbox",
]

workspace_root = Path("/content") / WORKSPACE_NAME
workspace_root.mkdir(parents=True, exist_ok=True)

globals()["WORKSPACE_ROOT"] = workspace_root
globals()["HF_MODELS"] = HF_MODELS
globals()["WORKSPACE_NAME"] = WORKSPACE_NAME

final_video_dir = workspace_root / "final_videos"
final_video_dir.mkdir(parents=True, exist_ok=True)
globals()["FINAL_VIDEO_DIR"] = str(final_video_dir)
os.environ["FINAL_VIDEO_DIR"] = str(final_video_dir)

model_notes_path = (REPO_ROOT / "docs" / "MODEL_INSTALL_NOTES.md").resolve()
if not model_notes_path.exists():
    raise FileNotFoundError(f"MODEL_INSTALL_NOTES not found at {model_notes_path}")

print(f"Configured workspace '{WORKSPACE_NAME}' → {workspace_root}")
print(f"Models to manage: {HF_MODELS or '[none specified]'}")
print(f"Model install reference: {model_notes_path}")
print(f"FINAL_VIDEO_DIR → {final_video_dir}")

env_path = REPO_ROOT / ".env"
if not env_path.exists():
    raise FileNotFoundError(
        f"No .env file found at {env_path}. Create it inside the repo checkout before running this cell."
    )

print(f"Using secrets file at {env_path}")

load_dotenv(env_path, override=True)
globals()["DOTENV_LOADED"] = True
print(f"Loaded environment variables from {env_path}")


Configured workspace 'SparkleMotion' → /content/SparkleMotion
Models to manage: ['stabilityai/stable-diffusion-xl-base-1.0', 'stabilityai/stable-diffusion-xl-refiner-1.0', 'Wan-AI/Wan2.1-I2V-14B-720P', 'Wan-AI/Wan2.1-FLF2V-14B-720P-diffusers', 'ResembleAI/chatterbox']
Model install reference: /content/sparkle_motion/docs/MODEL_INSTALL_NOTES.md
FINAL_VIDEO_DIR → /content/SparkleMotion/final_videos
Using secrets file at /content/sparkle_motion/.env
Loaded environment variables from /content/sparkle_motion/.env


## Auto-create artifacts + DB paths
Run this right after loading `.env`. It reads the `ARTIFACTS_DIR` and `SPARKLE_DB_PATH` values you just imported, ensures their parent directories exist, and creates the SQLite file if it is missing. Rerun anytime you change those paths.


In [ ]:
# Cell 3a: Ensure ARTIFACTS_DIR and SPARKLE_DB_PATH exist
import os
from pathlib import Path

if not globals().get("DOTENV_LOADED"):
    raise RuntimeError("Environment variables are missing. Run the .env loader first.")
if "REPO_ROOT" not in globals():
    raise RuntimeError("REPO_ROOT is undefined. Run the repo clone cell before continuing.")

repo_root = Path(globals()["REPO_ROOT"]).resolve()
env_path = repo_root / ".env"


def _require_absolute_env(key: str) -> Path:
    raw_value = os.environ.get(key, "").strip()
    if not raw_value:
        raise RuntimeError(f"{key} is missing or empty in {env_path}. Update the file and rerun the loader cell.")
    path = Path(raw_value).expanduser()
    if not path.is_absolute():
        raise RuntimeError(f"{key} must be an absolute path. Current value: {raw_value}")
    return path.resolve(strict=False)


def _ensure_directory(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    print(f"Ensured directory exists: {path}")


artifacts_dir = _require_absolute_env("ARTIFACTS_DIR")
_ensure_directory(artifacts_dir)

sparkle_db_path = _require_absolute_env("SPARKLE_DB_PATH")
_ensure_directory(sparkle_db_path.parent)
if sparkle_db_path.exists():
    print(f"SQLite registry already present at {sparkle_db_path}")
else:
    sparkle_db_path.touch()
    print(f"Initialized empty SQLite registry at {sparkle_db_path}")

os.environ["ARTIFACTS_DIR"] = str(artifacts_dir)
os.environ["SPARKLE_DB_PATH"] = str(sparkle_db_path)

print("Artifacts + database paths verified.")


Ensured directory exists: /content/SparkleMotion/artifacts
Ensured directory exists: /content/SparkleMotion
Initialized empty SQLite registry at /content/SparkleMotion/sparkle.db
Artifacts + database paths verified.


## Prepare workspace and download models
This cell does three things whenever you run it:
1. Calls `scripts/colab_drive_setup.py --workspace <WORKSPACE_ROOT>` (legacy name, still handles `/content` workspaces) to create any missing folders.
2. Downloads every repo listed in `HF_MODELS` (defined in the previous cell) using your `HF_TOKEN` from `.env`.
3. Writes a quick health report to `outputs/colab_smoke.json` so you can confirm each model pulled down correctly.

Before you press **Run**:
- Double-check the prior cell succeeded (so `WORKSPACE_ROOT`, `HF_MODELS`, and `DOTENV_LOADED` are all set).
- Replace entries in `HF_MODELS` with the exact Hugging Face repos you need (leave it empty only if you truly want to skip downloads).
- Make sure your `.env` includes `HF_TOKEN` if any repos are private.

Once those are ready, just run the next cell—it will stream the script output (progress + any Hugging Face errors) directly in the notebook.

In [ ]:
# Cell 4: Prepare workspace folders and download required models
from pathlib import Path
from typing import Sequence

from sparkle_motion.colab_helper import ensure_workspace

if not globals().get("DOTENV_LOADED"):
    raise RuntimeError("Environment variables are missing. Run the .env loader first.")
if "WORKSPACE_ROOT" not in globals():
    raise RuntimeError("Workspace is not configured. Run the workspace configuration cell first.")
if "HF_MODELS" not in globals():
    raise RuntimeError("HF_MODELS is missing. Set them in the workspace configuration cell.")
if "REPO_ROOT" not in globals():
    raise RuntimeError("REPO_ROOT is undefined. Run the repo clone cell first.")

workspace_root = Path(globals()["WORKSPACE_ROOT"]).resolve()
models: Sequence[str] = globals()["HF_MODELS"]
repo_root = Path(globals()["REPO_ROOT"]).resolve()

layout = ensure_workspace(workspace_root)
print(f"Workspace ready under {layout.root}")
print(f"models -> {layout.models}")


Workspace ready under /content/SparkleMotion
models -> /content/SparkleMotion/models
assets -> /content/SparkleMotion/assets
outputs -> /content/SparkleMotion/outputs



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 36 files:   0%|          | 0/36 [00:00<?, ?it/s]

text_encoder/model.onnx:   0%|          | 0.00/493M [00:00<?, ?B/s]

text_encoder/model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

(…)ion_pytorch_model.safetensors.index.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/508M [00:00<?, ?B/s]

Fetching 0 files: 0it [00:00, ?it/s]


Smoke report written to /content/SparkleMotion/outputs/colab_smoke.json
Workspace ready and all models downloaded.


## Install Wav2Lip (lipsync helper)
This step vendors the upstream [Wav2Lip](https://github.com/Rudrabha/Wav2Lip) repo into your workspace, installs its Python requirements, and downloads the `wav2lip_gan.pth` checkpoint the lipsync stage needs. Run it after the Hugging Face download cell so the workspace already exists.

The cell below will:
- Clone (or update) the Wav2Lip repo under your `WORKSPACE_ROOT`.
- Install `requirements.txt` with `pip`.
- Fetch `wav2lip_gan.pth` into `Wav2Lip/checkpoints/`.

It also exports `WAV2LIP_ROOT` so downstream helpers pick up the local path automatically.

In [10]:
!cd /content/SparkleMotion && git clone https://github.com/Rudrabha/Wav2Lip.git && \
    cd Wav2Lip/checkpoints && pip install -r requirements.txt && \
    wget  https://huggingface.co/Rudrabha/Wav2Lip/resolve/main/wav2lip_gan.pth

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 409 (delta 2), reused 0 (delta 0), pack-reused 405 (from 2)
Receiving objects: 100% (409/409), 549.05 KiB | 5.84 MiB/s, done.
Resolving deltas: 100% (227/227), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


## Colab preflight helper
Right after the installs finish, run the next cell to make sure this runtime is ready to go.

- Press **Run** and wait a few seconds.
- It checks that your `.env` loaded, `/content` is writable, you have enough disk/GPU, and the helper servers respond.
- You’ll get a short list with ✅/❌ results. If a line fails, the message tells you what to fix; rerun the cell once it’s resolved.

## Start the local servers before preflight
Use these buttons right after the installs finish. Start ScriptAgent (port 5001) and ProductionAgent (port 5008) here so the preflight checks and control panel have live FastAPI endpoints. Stop them when you wrap up the session.


In [11]:
import os
import signal
import subprocess
import sys
from pathlib import Path
from typing import Dict, NamedTuple

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError as exc:  # pragma: no cover - notebook utility guard
    raise RuntimeError("ipywidgets is required for the Workflow Agent controls") from exc


class ServerProcess(NamedTuple):
    process: subprocess.Popen
    log_path: Path
    log_handle: object


SERVER_CONFIGS: Dict[str, Dict[str, object]] = {
    "script_agent": {
        "app": "sparkle_motion.function_tools.script_agent.entrypoint:app",
        "port": 5001,
    },
    "production_agent": {
        "app": "sparkle_motion.function_tools.production_agent.entrypoint:app",
        "port": 5008,
    },
}

RUNNING_SERVERS: Dict[str, ServerProcess] = {}

PYTHONPATH = os.pathsep.join(
    filter(None, {str(Path(REPO_ROOT)), str(Path(REPO_ROOT) / "src"), os.environ.get("PYTHONPATH", "")})
)


def _server_cmd(name: str) -> list[str]:
    cfg = SERVER_CONFIGS[name]
    return [
        sys.executable,
        "-m",
        "uvicorn",
        cfg["app"],
        "--host",
        os.environ.get("WORKFLOW_AGENT_HOST", "127.0.0.1"),
        "--port",
        str(cfg["port"]),
        "--no-access-log",
    ]


def start_server(name: str) -> None:
    server = RUNNING_SERVERS.get(name)
    if server and server.process.poll() is None:
        raise RuntimeError(f"{name} already running (pid={server.process.pid})")
    env = os.environ.copy()
    env["PYTHONPATH"] = PYTHONPATH
    log_path = Path(REPO_ROOT) / "tmp" / f"{name}.log"
    log_path.parent.mkdir(parents=True, exist_ok=True)
    log_handle = open(log_path, "ab", buffering=0)
    proc = subprocess.Popen(
        _server_cmd(name),
        env=env,
        stdout=log_handle,
        stderr=subprocess.STDOUT,
    )
    RUNNING_SERVERS[name] = ServerProcess(process=proc, log_path=log_path, log_handle=log_handle)
    print(f"Started {name} on port {SERVER_CONFIGS[name]['port']} (pid={proc.pid}). Logs → {log_path}")


def stop_server(name: str, *, sig: int = signal.SIGTERM) -> None:
    server = RUNNING_SERVERS.get(name)
    if not server:
        print(f"{name} has not been started from this notebook.")
        return
    proc = server.process
    if proc.poll() is not None:
        print(f"{name} already stopped (pid={proc.pid}).")
    else:
        proc.send_signal(sig)
        try:
            proc.wait(timeout=10)
        except subprocess.TimeoutExpired:
            proc.kill()
            proc.wait()
        print(f"Stopped {name} (pid={proc.pid}).")
    server.log_handle.close()
    RUNNING_SERVERS.pop(name, None)


def list_servers() -> Dict[str, str]:
    status = {}
    for name in SERVER_CONFIGS:
        proc = RUNNING_SERVERS.get(name)
        if proc and proc.process.poll() is None:
            status[name] = f"running (pid={proc.process.pid})"
        else:
            status[name] = "stopped"
    return status


server_selector = widgets.Dropdown(options=list(SERVER_CONFIGS.keys()), description="Server")
start_button = widgets.Button(description="Start", button_style="success")
stop_button = widgets.Button(description="Stop", button_style="danger")
status_button = widgets.Button(description="Status", button_style="info")
output = widgets.Output()


def _handle_start(_: widgets.Button) -> None:
    with output:
        output.clear_output()
        try:
            start_server(server_selector.value)
        except Exception as exc:  # pragma: no cover - notebook UX guard
            print(f"Failed to start {server_selector.value}: {exc}")


def _handle_stop(_: widgets.Button) -> None:
    with output:
        output.clear_output()
        stop_server(server_selector.value)


def _handle_status(_: widgets.Button) -> None:
    with output:
        output.clear_output()
        for name, state in list_servers().items():
            print(f"{name}: {state}")


start_button.on_click(_handle_start)
stop_button.on_click(_handle_stop)
status_button.on_click(_handle_status)

controls = widgets.HBox([server_selector, start_button, stop_button, status_button])
display(widgets.VBox([controls, output]))


In [12]:
# Cell 6: Run the consolidated Colab preflight checks
from pathlib import Path
from sparkle_motion.notebook_preflight import format_report, run_preflight_checks

if "WORKSPACE_ROOT" not in globals():
    raise RuntimeError("WORKSPACE_ROOT is undefined. Run the workspace configuration cell first.")
if "REPO_ROOT" not in globals():
    raise RuntimeError("REPO_ROOT is undefined. Run the repo clone cell first.")

workspace_root = Path(globals()["WORKSPACE_ROOT"]).resolve()
repo_root = Path(globals()["REPO_ROOT"]).resolve()

preflight_results = run_preflight_checks(
    requirements_path=repo_root / "requirements-ml.txt",
    mount_point=workspace_root,
    workspace_dir=workspace_root,
    ready_endpoints=(
        "http://localhost:5001/ready",
        "http://localhost:5008/ready",
    ),
    pip_mode="install",
    require_drive=False,
    skip_gpu_checks=False,
)

print(format_report(preflight_results))

[OK     ] env: All required env vars present.
[OK     ] pip: Dependencies installed via requirements-ml.txt.
[WARNING] drive: Drive mount not required (flag disabled).
[OK     ] disk: 61.8 GiB free at /content/SparkleMotion.
[OK     ] gpu: nvidia-smi detected; GPU is attached.
[OK     ] ready: Probed 2 /ready endpoints.


In [13]:
# Cell 6b: Inspect smoke artifact with per-model status
import json
from pathlib import Path

if "WORKSPACE_ROOT" not in globals():
    raise RuntimeError("WORKSPACE_ROOT is undefined. Run the workspace configuration cell first.")

workspace_root = Path(globals()["WORKSPACE_ROOT"]).resolve()
smoke_path = workspace_root / "outputs" / "colab_smoke.json"
if smoke_path.exists():
    data = json.loads(smoke_path.read_text(encoding="utf-8"))
    status = "OK" if data.get("ok") else "FAILED"
    print(f"Smoke status: {status}")
    for model in data.get("models", []):
        sample = model.get("sample_file") or "n/a"
        print(
            f"- {model['repo_id']}: {model['status']} "
            f"({model.get('files_present', 0)} files, sample={sample})"
        )
else:
    print("Smoke artifact not found yet — run the helper above.")

Smoke status: OK
- stabilityai/stable-diffusion-xl-base-1.0: present (109 files, sample=/content/SparkleMotion/models/stabilityai__stable-diffusion-xl-base-1.0/tokenizer/special_tokens_map.json)
- stabilityai/stable-diffusion-xl-refiner-1.0: present (52 files, sample=/content/SparkleMotion/models/stabilityai__stable-diffusion-xl-refiner-1.0/vae_1_0/config.json)
- Wan-AI/Wan2.1-I2V-14B-720P: present (25 files, sample=/content/SparkleMotion/models/Wan-AI__Wan2.1-I2V-14B-720P/xlm-roberta-large/special_tokens_map.json)
- Wan-AI/Wan2.1-FLF2V-14B-720P-diffusers: present (115 files, sample=/content/SparkleMotion/models/Wan-AI__Wan2.1-FLF2V-14B-720P-diffusers/image_processor/special_tokens_map.json)
- ResembleAI/chatterbox: present (0 files, sample=n/a)


## Artifact Service controls
Use these buttons when you want the notebook to save artifacts straight to your `/content/<workspace_name>` workspace without talking to the hosted service.

1. **Show settings** – prints the environment variables (paths, token) so you can reuse them in another shell if needed.
2. **Start service** – launches the local Artifact Service in the background so stages can read/write files right away.
3. **Check health** – confirms the service is responding before you point anything at it.

Start it once per session, then leave it running until you’re done. Everything lands inside your `/content/<workspace_name>` tree so runs stay self-contained in this runtime.

In [15]:
import os
import signal
import subprocess
import sys
from pathlib import Path
from urllib.parse import urlparse

try:
    import ipywidgets as widgets
    from IPython.display import display
except ImportError as exc:  # pragma: no cover - notebook helper
    raise RuntimeError("ipywidgets is required for the filesystem shim controls") from exc

from sparkle_motion.filesystem_artifacts.config import DEFAULT_ARTIFACTS_FS_BASE_URL

if "WORKSPACE_ROOT" not in globals():
    raise RuntimeError("WORKSPACE_ROOT is undefined. Run the workspace configuration cell first.")

workspace_root = Path(globals()["WORKSPACE_ROOT"]).resolve()
REPO_ROOT = Path(os.environ.get("SPARKLE_MOTION_REPO_ROOT", Path.cwd())).resolve()
SRC_PATH = REPO_ROOT / "src"
_base_url = os.environ.get("ARTIFACTS_FS_BASE_URL", DEFAULT_ARTIFACTS_FS_BASE_URL)
_parsed_base = urlparse(_base_url)
_default_host = _parsed_base.hostname or "127.0.0.1"
_default_port = _parsed_base.port or (443 if _parsed_base.scheme == "https" else 80)
FS_SHIM_HOST = os.environ.get("FILESYSTEM_SHIM_HOST", _default_host)
FS_SHIM_PORT = int(os.environ.get("FILESYSTEM_SHIM_PORT") or _default_port)
FS_ROOT = (workspace_root / "artifacts_fs").resolve()
FS_ROOT.mkdir(parents=True, exist_ok=True)
_shim_process: subprocess.Popen | None = None
_shim_log_handle = None
_shim_log_path = REPO_ROOT / "tmp" / "filesystem_shim.log"
_shim_log_path.parent.mkdir(parents=True, exist_ok=True)


def _shim_env() -> dict[str, str]:
    env = os.environ.copy()
    env.setdefault("ARTIFACTS_BACKEND", "filesystem")
    env.setdefault("ARTIFACTS_FS_ROOT", str(FS_ROOT))
    env.setdefault("ARTIFACTS_FS_INDEX", str(FS_ROOT / "index.db"))
    env.setdefault("ARTIFACTS_FS_BASE_URL", f"http://{FS_SHIM_HOST}:{FS_SHIM_PORT}")
    env.setdefault("ARTIFACTS_FS_TOKEN", env.get("ARTIFACTS_FS_TOKEN") or "local-fs-token")
    env["PYTHONPATH"] = os.pathsep.join(filter(None, {str(REPO_ROOT), str(SRC_PATH), env.get("PYTHONPATH", "")}))
    return env


def _start_filesystem_shim(_: widgets.Button | None = None) -> None:
    global _shim_process, _shim_log_handle
    if _shim_process and _shim_process.poll() is None:
        with shim_output:
            shim_output.clear_output()
            print(f"Filesystem shim already running on {FS_SHIM_HOST}:{FS_SHIM_PORT} (pid={_shim_process.pid}).")
        return
    env = _shim_env()
    _shim_log_handle = open(_shim_log_path, "ab", buffering=0)
    cmd = [
        sys.executable,
        "scripts/filesystem_artifacts.py",
        "serve",
        "--host",
        FS_SHIM_HOST,
        "--port",
        str(FS_SHIM_PORT),
    ]
    _shim_process = subprocess.Popen(
        cmd,
        env=env,
        stdout=_shim_log_handle,
        stderr=subprocess.STDOUT,
        cwd=str(REPO_ROOT),
    )
    with shim_output:
        shim_output.clear_output()
        print(f"Started filesystem shim on {FS_SHIM_HOST}:{FS_SHIM_PORT} (pid={_shim_process.pid}).")
        print(f"Logs → {_shim_log_path}")


def _stop_filesystem_shim(_: widgets.Button | None = None) -> None:
    global _shim_process, _shim_log_handle
    if not _shim_process:
        with shim_output:
            shim_output.clear_output()
            print("Filesystem shim has not been started from this notebook.")
        return
    if _shim_process.poll() is None:
        _shim_process.send_signal(signal.SIGTERM)
        try:
            _shim_process.wait(timeout=10)
        except subprocess.TimeoutExpired:
            _shim_process.kill()
            _shim_process.wait()
    if _shim_log_handle:
        _shim_log_handle.close()
        _shim_log_handle = None
    with shim_output:
        shim_output.clear_output()
        print("Filesystem shim stopped.")
    _shim_process = None


def _shim_status(_: widgets.Button | None = None) -> None:
    with shim_output:
        shim_output.clear_output()
        if _shim_process and _shim_process.poll() is None:
            print(f"Running on {FS_SHIM_HOST}:{FS_SHIM_PORT} (pid={_shim_process.pid}).")
        else:
            print("Filesystem shim is stopped.")


def _print_env_exports(_: widgets.Button | None = None) -> None:
    env = _shim_env()
    cmd = [
        sys.executable,
        "scripts/filesystem_artifacts.py",
        "env",
        "--shell",
        "bash",
        "--root",
        env["ARTIFACTS_FS_ROOT"],
        "--index",
        env["ARTIFACTS_FS_INDEX"],
        "--token",
        env["ARTIFACTS_FS_TOKEN"],
        "--emit-token",
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, env=env, cwd=str(REPO_ROOT))
    with env_output:
        env_output.clear_output()
        if result.returncode == 0:
            print(result.stdout.strip())
        else:
            print(result.stderr or result.stdout)


def _health_probe(_: widgets.Button | None = None) -> None:
    env = _shim_env()
    cmd = [
        sys.executable,
        "scripts/filesystem_artifacts.py",
        "health",
        "--url",
        f"http://{FS_SHIM_HOST}:{FS_SHIM_PORT}/healthz",
        "--token",
        env["ARTIFACTS_FS_TOKEN"],
    ]
    result = subprocess.run(cmd, capture_output=True, text=True, env=env, cwd=str(REPO_ROOT))
    with env_output:
        env_output.clear_output()
        if result.returncode == 0:
            print(result.stdout.strip())
        else:
            print(result.stderr or result.stdout)


start_button = widgets.Button(description="Start shim", button_style="success")
stop_button = widgets.Button(description="Stop shim", button_style="danger")
status_button = widgets.Button(description="Status", button_style="info")
env_button = widgets.Button(description="Show env exports")
health_button = widgets.Button(description="Probe /healthz", icon="heartbeat")
shim_output = widgets.Output()
env_output = widgets.Output()

start_button.on_click(_start_filesystem_shim)
stop_button.on_click(_stop_filesystem_shim)
status_button.on_click(_shim_status)
env_button.on_click(_print_env_exports)
health_button.on_click(_health_probe)

controls_row = widgets.HBox([start_button, stop_button, status_button, env_button, health_button])
display(widgets.VBox([controls_row, shim_output, env_output]))

## Control panel: type your prompt here
This is the UI you actually use to make a movie. When you run the next cell it pops up a panel with two text boxes: **Title** and **Prompt**. Type whatever story prompt you want in that Prompt box.

1. Type your idea in the Prompt box (add a short title if you want).
2. Click **Generate Plan**. That sends your prompt to ScriptAgent and shows the plan it creates.
3. Set the Mode dropdown to `Run` and click **Run Production**. That hands the plan to ProductionAgent so it can build the video.

The panel keeps the latest run ID so the later cells (status, artifacts, final video download) know what to show. Nothing else in this notebook collects your prompt, so always start here when you want Sparkle Motion to generate a film.

In [16]:
from pathlib import Path
import os
import sys

REPO_ROOT = Path(os.environ.get("SPARKLE_MOTION_REPO_ROOT", Path.cwd())).resolve()
globals()["REPO_ROOT"] = REPO_ROOT
if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))
SRC_PATH = REPO_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

In [17]:
# Quickstart cell: import and display the ipywidgets control panel
from notebooks.control_panel import create_control_panel

print("Launching control panel with endpoints from configs/tool_registry.yaml (profile='local-colab').")
control_panel = create_control_panel()
control_panel

Launching control panel with endpoints from configs/tool_registry.yaml (profile='local-colab').
[control_panel] Logging events to /content/sparkle_motion/artifacts/logs/control_panel_20251204_074033.log


## Final video preview & download
Finished a run? Use this helper to grab the final MP4. It pulls the finished video, shows it right in the notebook, and, if you’re on Colab, pops up the download dialog so you can save it locally.

In [ ]:
# Optional: override FINAL_VIDEO_DIR after changing workspace layout
import os
from pathlib import Path

if "WORKSPACE_ROOT" not in globals():
    raise RuntimeError("WORKSPACE_ROOT is undefined. Run the workspace configuration cell first.")

CUSTOM_FINAL_VIDEO_SUBDIR = "final_videos"  # change to another folder name if desired
workspace_root = Path(globals()["WORKSPACE_ROOT"]).resolve()
final_dir = (workspace_root / CUSTOM_FINAL_VIDEO_SUBDIR).resolve()
final_dir.mkdir(parents=True, exist_ok=True)
os.environ["FINAL_VIDEO_DIR"] = str(final_dir)
print(f"FINAL_VIDEO_DIR reset to {final_dir}")

In [ ]:
# Cell 5: Final deliverable helper (finalize)
import importlib
import os
import subprocess
from pathlib import Path
from typing import Any

import httpx
from IPython.display import HTML, display

from notebooks import preview_helpers
from sparkle_motion import tool_registry


def _production_agent_base() -> str:
    env_override = os.environ.get("PRODUCTION_AGENT_BASE")
    if env_override:
        return env_override
    info = tool_registry.get_local_endpoint_info("production_agent")
    if info:
        return info.base_url
    return "http://127.0.0.1:5008"


PRODUCTION_AGENT_BASE = _production_agent_base()
FINALIZE_STAGE = "finalize"
DOWNLOAD_DIR = Path(os.environ.get("FINAL_VIDEO_DIR", "/content/final_videos"))
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

colab_files = None
try:
    _colab_spec = importlib.util.find_spec("google.colab.files")
except ModuleNotFoundError:
    _colab_spec = None
if _colab_spec:
    colab_files = importlib.import_module("google.colab.files")


def _current_run_id() -> str:
    if "control_panel" in globals():
        cp = globals()["control_panel"]
        run_value = getattr(getattr(cp, "run_id_input", None), "value", "")
        if run_value and run_value.strip():
            return run_value.strip()
        state = getattr(cp, "state", None)
        if state and getattr(state, "last_run_request_id", None):
            return state.last_run_request_id
    return os.environ.get("RUN_ID", "").strip()


RUN_ID = _current_run_id()
if not RUN_ID:
    raise RuntimeError(
        "Set RUN_ID (or populate control_panel.run_id_input) before running the final deliverable helper.",
    )


def _fetch_stage_manifest(run_id: str) -> dict[str, Any]:
    stage_manifest = preview_helpers.fetch_stage_manifest(
        base_url=PRODUCTION_AGENT_BASE,
        run_id=run_id,
        stage=FINALIZE_STAGE,
    )
    summary = preview_helpers.render_stage_summary(stage_manifest)
    print(summary)
    finalize_status = stage_manifest.get("status") or stage_manifest.get("state")
    if finalize_status:
        print(f"Finalize status => {finalize_status}")
    return stage_manifest


def _locate_video_final(stage_manifest: dict[str, Any]) -> dict[str, Any]:
    for entry in stage_manifest.get("artifacts") or []:
        if entry.get("artifact_type") == "video_final":
            return entry
    raise RuntimeError(
        "No video_final artifact found. Ensure finalize succeeded or resume production_agent with resume_from='finalize'.",
    )


def ensure_local_video(entry: dict[str, Any], run_id: str) -> Path:
    local_path = entry.get("local_path")
    if local_path:
        candidate = Path(local_path).expanduser()
        if candidate.exists():
            return candidate
    target = DOWNLOAD_DIR / f"{run_id}_video_final.mp4"
    download_url = entry.get("download_url")
    if download_url:
        with httpx.Client(timeout=None) as client:
            with client.stream("GET", download_url) as resp:
                resp.raise_for_status()
                with target.open("wb") as handle:
                    for chunk in resp.iter_bytes():
                        handle.write(chunk)
        return target
    artifact_uri = entry.get("artifact_uri")
    if artifact_uri:
        result = subprocess.run(
            ["adk", "artifacts", "download", artifact_uri, str(target)],
            capture_output=True,
            text=True,
            check=False,
        )
        if result.returncode == 0 and target.exists():
            return target
        print("ADK artifact download failed:")
        print(result.stderr or result.stdout)
    raise RuntimeError(
        "Unable to download the final video locally. Provide download_url or local_path in the artifacts manifest.",
    )


def render_preview_video(preview_entry: dict[str, Any] | None) -> None:
    if not preview_entry:
        print("No preview clip available from /artifacts preview metadata.")
        return
    local_path = preview_entry.get("local_path")
    if local_path and Path(local_path).exists():
        display(HTML("<strong>Preview clip</strong>"))
        preview_widget = preview_helpers.create_video_widget(
            {"local_path": local_path, "artifact_type": "preview_clip"},
            width=480,
        )
        display(preview_widget)
        return
    download_url = preview_entry.get("download_url")
    if download_url:
        print(f"Preview available remotely (download_url={download_url}).")
    else:
        print("Preview metadata present but no local path or download URL; see artifact_uri for details.")


stage_manifest = _fetch_stage_manifest(RUN_ID)
preview_entry = (stage_manifest.get("preview") or {}).get("video")
final_entry = _locate_video_final(stage_manifest)

render_preview_video(preview_entry)
video_path = ensure_local_video(final_entry, RUN_ID)
final_entry["local_path"] = str(video_path)

info_html = f"""
<p><strong>Run ID:</strong> {RUN_ID}</p>
<p><strong>Artifact URI:</strong> {final_entry.get('artifact_uri', 'n/a')}</p>
<p><strong>Local path:</strong> {video_path}</p>
"""
display(HTML(info_html))

display(preview_helpers.create_video_widget(final_entry, width=640))

if colab_files is not None:
    colab_files.download(str(video_path))
else:
    print("Download helper available only inside Google Colab. Share the path above manually if running elsewhere.")

## Artifacts viewer
Use this helper to inspect `/artifacts` responses without leaving the notebook. It shares the `control_panel` run metadata when available, lets you scope by stage (e.g., `finalize`), and can poll automatically so new artifacts appear as production advances.

In [ ]:
# Cell 4c: Artifacts viewer helper
import asyncio
import json
import os
from typing import Any as ArtifactAny, Dict as ArtifactDict

import httpx as httpx_client
import ipywidgets as widgets
from IPython.display import HTML, display

from notebooks import preview_helpers
from sparkle_motion import tool_registry as artifact_tool_registry

ArtifactPayload = ArtifactDict[str, ArtifactAny]


def _production_agent_base() -> str:
    env_override = os.environ.get("PRODUCTION_AGENT_BASE")
    if env_override:
        return env_override
    info = artifact_tool_registry.get_local_endpoint_info("production_agent")
    if info:
        return info.base_url
    return "http://127.0.0.1:5008"


PRODUCTION_AGENT_BASE = _production_agent_base()
ARTIFACTS_ENDPOINT = f"{PRODUCTION_AGENT_BASE}/artifacts"

if "artifact_viewer_state" in globals():
    existing_task = globals()["artifact_viewer_state"].get("task")
    if existing_task and not existing_task.done():
        existing_task.cancel()

artifact_viewer_state: ArtifactDict[str, ArtifactAny] = {"task": None}


def _artifact_viewer_run_id() -> str:
    if "control_panel" in globals():
        cp = globals()["control_panel"]
        run_widget = getattr(cp, "run_id_input", None)
        candidate = getattr(run_widget, "value", "")
        if candidate and candidate.strip():
            return candidate.strip()
        state = getattr(cp, "state", None)
        if state and getattr(state, "last_run_request_id", None):
            return state.last_run_request_id
    return os.environ.get("RUN_ID", "").strip()


def _format_status_html(message: str, *, ok: bool) -> str:
    color = "#3c763d" if ok else "#d9534f"
    return f"<span style='color:{color}; font-size:0.9em;'>{message}</span>"


def _iter_artifacts(payload: ArtifactPayload):
    stages = payload.get("stages") or []
    for stage in stages:
        for artifact in stage.get("artifacts") or []:
            yield artifact
    for artifact in payload.get("artifacts") or []:
        yield artifact


def _render_artifacts(payload: ArtifactPayload) -> None:
    artifacts = list(_iter_artifacts(payload))
    stages = payload.get("stages") or []
    with artifacts_output:
        artifacts_output.clear_output()
        print(f"Artifacts returned: {len(artifacts)}")
        if stages:
            for stage in stages:
                stage_label = stage.get("stage") or stage.get("stage_id") or "stage"
                print(f"\nStage: {stage_label}")
                print(preview_helpers.render_stage_summary(stage))
                previewable = [entry for entry in (stage.get("artifacts") or []) if entry.get("local_path")]
                if previewable:
                    preview_helpers.display_artifact_previews(
                        {**stage, "artifacts": previewable},
                        max_items=4,
                        video_width=360,
                    )
                else:
                    print("Local previews unavailable yet; see raw payload below.")
        else:
            print("No stage sections returned; dumping raw payload.")
        print("\nFull payload:\n")
        print(json.dumps(payload, indent=2, ensure_ascii=False))


def _fetch_artifacts_sync(run_id: str, stage: str) -> ArtifactPayload:
    params = {"run_id": run_id}
    if stage:
        params["stage"] = stage
    with httpx_client.Client(timeout=30.0) as client:
        resp = client.get(ARTIFACTS_ENDPOINT, params=params)
        resp.raise_for_status()
        data = resp.json()
        if not isinstance(data, dict):
            raise RuntimeError("Unexpected artifacts response payload")
        return data


async def _fetch_artifacts_async(client: httpx_client.AsyncClient, run_id: str, stage: str) -> ArtifactPayload:
    params = {"run_id": run_id}
    if stage:
        params["stage"] = stage
    resp = await client.get(ARTIFACTS_ENDPOINT, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not isinstance(data, dict):
        raise RuntimeError("Unexpected artifacts response payload")
    return data


async def _poll_status(run_id: str, stage: str, interval_s: float, state: ArtifactDict[str, ArtifactAny]) -> None:
    async with httpx_client.AsyncClient(timeout=30.0) as client:
        while True:
            try:
                payload = await _fetch_artifacts_async(client, run_id, stage)
            except Exception as exc:  # pragma: no cover - notebook UX guard
                with artifacts_output:
                    artifacts_output.clear_output()
                    print(f"Artifacts request failed: {exc}")
            else:
                _render_artifacts(payload)
            await asyncio.sleep(interval_s)
            if state.get("task") is None or state.get("cancel"):
                break


def _handle_fetch(_: widgets.Button | None = None) -> None:
    run_id = run_id_input.value.strip() or _artifact_viewer_run_id()
    stage = stage_input.value.strip()
    if not run_id:
        with status_output:
            status_output.clear_output()
            print("Populate RUN_ID first (control panel or env).")
        return
    try:
        payload = _fetch_artifacts_sync(run_id, stage)
    except Exception as exc:  # pragma: no cover - notebook UX guard
        with status_output:
            status_output.clear_output()
            print(f"Artifacts request failed: {exc}")
        return
    with status_output:
        status_output.clear_output()
        print(_format_status_html("Artifacts fetched", ok=True))
    _render_artifacts(payload)


def _handle_manual_refresh(_: widgets.Button | None = None) -> None:
    _handle_fetch(None)


def _handle_poll_change(change: ArtifactDict[str, ArtifactAny]) -> None:
    enabled = change.get("new") is True
    run_id = run_id_input.value.strip() or _artifact_viewer_run_id()
    stage = stage_input.value.strip()
    if enabled:
        if not run_id:
            with status_output:
                status_output.clear_output()
                print(_format_status_html("Populate RUN_ID before polling", ok=False))
            poll_toggle.value = False
            return
        state = artifact_viewer_state
        state["cancel"] = False
        task = asyncio.create_task(
            _poll_status(run_id, stage, interval_input.value, state),
        )
        state["task"] = task
        with status_output:
            status_output.clear_output()
            print(_format_status_html("Polling started", ok=True))
    else:
        state = artifact_viewer_state
        state["cancel"] = True
        task = state.get("task")
        if task:
            task.cancel()
            state["task"] = None
        with status_output:
            status_output.clear_output()
            print(_format_status_html("Polling stopped", ok=True))


run_id_input = widgets.Text(description="Run ID", placeholder="Auto from control panel")
stage_input = widgets.Text(description="Stage", placeholder="finalize")
interval_input = widgets.BoundedFloatText(value=5.0, min=2.0, max=60.0, description="Interval (s)")
fetchnow_button = widgets.Button(description="Fetch once", button_style="primary")
poll_toggle = widgets.ToggleButton(description="Poll", icon="refresh", value=False)
status_output = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", min_height="50px"))
artifacts_output = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", min_height="160px", max_height="320px", overflow="auto"))

fetchnow_button.on_click(_handle_fetch)
poll_toggle.observe(_handle_poll_change, names="value")

controls = widgets.HBox([run_id_input, stage_input, fetchnow_button, poll_toggle, interval_input])

viewer = widgets.VBox([
    controls,
    status_output,
    artifacts_output,
])

display(viewer)


### Quick artifacts viewer helpers
Use the next three cells when you need to debug the artifacts viewer:
1. **Sync run id** — copies the current Run ID from the control panel into the viewer so both panels match.
2. **Manual refresh** — triggers one fetch of the artifacts list so you can see the latest verification logs without waiting for auto-refresh.
3. **Snapshot payload** — prints a JSON summary (run id, artifact count, stage names) for your notebook log or troubleshooting notes.
Only run the ones you need; each prints its own confirmation when it completes.

In [23]:
# Sync artifacts viewer Run ID widget with control panel
if "control_panel" in globals() and hasattr(control_panel, "run_id_input"):
    run_id_input.value = control_panel.run_id_input.value
print("Artifacts viewer run_id now:", run_id_input.value)


Artifacts viewer run_id now: run_53ac658ea895


In [24]:
# Trigger a manual artifacts refresh for verification logs
_handle_manual_refresh(None)
print("Artifacts viewer manual refresh invoked.")


NameError: name '_handle_manual_refresh' is not defined

In [25]:
# Capture artifacts payload for notebook log (single fetch)
import json
current_run = run_id_input.value.strip() or _artifact_viewer_run_id()
payload_snapshot = _fetch_artifacts_sync(current_run, stage_input.value.strip())
stage_names = []
for stage in payload_snapshot.get("stages", []):
    stage_names.append(stage.get("stage") or stage.get("stage_id"))
print(json.dumps(
    {
        "run_id": current_run,
        "artifact_count": len(list(_iter_artifacts(payload_snapshot))),
        "stages": stage_names,
    },
    indent=2,
    ensure_ascii=False,
))


{
  "run_id": "run_53ac658ea895",
  "artifact_count": 16,
  "stages": [
    "plan_intake",
    "assemble",
    "finalize"
  ]
}


## Filesystem cleanup helper
Run this cell and hit **Run prune** if you need to clear out old artifacts.

- Tweak the keep limits (max size, age, or optional run IDs) if you want to be selective.
- The cleanup log streams right under the form so you can watch what gets removed.

In [ ]:
import os
import shlex
import subprocess
import sys
from pathlib import Path

import ipywidgets as widgets

REPO_ROOT = Path.cwd().resolve()
CLI_PATH = REPO_ROOT / "scripts" / "filesystem_artifacts.py"

ROOT_VALUE = os.environ.get("ARTIFACTS_FS_ROOT", "").strip()
INDEX_VALUE = os.environ.get("ARTIFACTS_FS_INDEX", "").strip()
backend_value = os.environ.get("ARTIFACTS_BACKEND", "")

max_bytes_input = widgets.Text(value="200g", description="Max bytes")
max_age_input = widgets.BoundedFloatText(value=14.0, min=0.0, max=365.0, step=1.0, description="Max age (days)")
min_free_input = widgets.Text(value="", description="Min free")
runs_input = widgets.Textarea(
    value="",
    placeholder="Optional run IDs (one per line)",
    description="Run filter",
    layout=widgets.Layout(width="60%", height="80px"),
)
assume_yes_checkbox = widgets.Checkbox(value=False, description="Auto confirm (--yes)", indent=False)
run_button = widgets.Button(description="Run prune", icon="trash")
status_label = widgets.HTML("")
log_output = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", max_height="260px", overflow="auto"))


def _format_status(message: str, *, ok: bool) -> str:
    color = "#2e7d32" if ok else "#c62828"
    return f"<span style='color:{color}; font-weight:bold'>{message}</span>"


def _build_command() -> list[str]:
    cmd = [sys.executable, str(CLI_PATH), "prune"]
    if ROOT_VALUE:
        cmd.extend(["--root", ROOT_VALUE])
    if INDEX_VALUE:
        cmd.extend(["--index", INDEX_VALUE])
    if max_bytes_input.value.strip():
        cmd.extend(["--max-bytes", max_bytes_input.value.strip()])
    if max_age_input.value and max_age_input.value > 0:
        cmd.extend(["--max-age-days", str(max_age_input.value)])
    if min_free_input.value.strip():
        cmd.extend(["--min-free-bytes", min_free_input.value.strip()])
    for run_id in runs_input.value.splitlines():
        run_id = run_id.strip()
        if run_id:
            cmd.extend(["--run", run_id])
    cmd.append("--no-dry-run")
    if assume_yes_checkbox.value:
        cmd.append("--yes")
    return cmd


def _run_prune(_btn: widgets.Button) -> None:
    log_output.clear_output()
    errors = []
    effective_backend = (backend_value or os.environ.get("ARTIFACTS_BACKEND", "")).lower()
    if effective_backend != "filesystem":
        errors.append("Set ARTIFACTS_BACKEND=filesystem before pruning.")
    if not CLI_PATH.exists():
        errors.append(f"Missing CLI script: {CLI_PATH}")
    if not (max_bytes_input.value.strip() or max_age_input.value > 0 or min_free_input.value.strip()):
        errors.append("Provide at least one retention constraint (max bytes / age / min free).")
    if errors:
        status_label.value = _format_status(" ".join(errors), ok=False)
        return

    cmd = _build_command()
    env = os.environ.copy()
    pythonpath_bits = [str(REPO_ROOT), str(REPO_ROOT / "src")]
    if env.get("PYTHONPATH"):
        pythonpath_bits.append(env["PYTHONPATH"])
    env["PYTHONPATH"] = os.pathsep.join(pythonpath_bits)

    status_label.value = _format_status("Running prune command…", ok=True)

    with log_output:
        print("$", " ".join(shlex.quote(part) for part in cmd))
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)
        assert process.stdout is not None
        for line in process.stdout:
            print(line.rstrip())
        return_code = process.wait()

    if return_code == 0:
        status_label.value = _format_status("Prune command completed.", ok=True)
    else:
        status_label.value = _format_status(f"Prune command failed (exit {return_code}).", ok=False)


run_button.on_click(_run_prune)

prune_controls = widgets.VBox(
    [
        widgets.HBox([max_bytes_input, max_age_input, min_free_input]),
        runs_input,
        widgets.HBox([assume_yes_checkbox, run_button]),
        status_label,
        log_output,
    ]
)

if ROOT_VALUE and INDEX_VALUE:
    status_label.value = _format_status("Ready: current workspace paths loaded from env.", ok=True)

display(prune_controls)
